In [ ]:
import os

if not os.path.exists('./templates'): 
    os.makedirs('./templates')
    
if not os.path.exists('./static'):
    os.makedirs('./static')

In [15]:
%%file hello.py
from flask import Flask, request

app = Flask(__name__)

@app.route('/')
def welcome():
    return 'This is root page'

@app.route('/hello')
def hello():
    """Return a friendly HTTP greeting"""
    print('I am inside hello world')
    return 'Hello World!'

@app.route('/echo/<name>')
def echo(name):
    """echo <name>"""
    return f'Hello {name}'

@app.route('/method', methods=['GET', 'POST'])
def method():
    if request.method == 'POST':
        return "You've used a Post request."
    else:
        return "I reckon you're probally using a GET request."

@app.route('/query-example')
def query_example():
    language = request.args.get('language')
    return '''<h1>The language value is: {} </h1>'''.format(language)
  
if __name__=='__main__':
    app.run(host='0.0.0.0', debug=True)

Writing hello.py


In [2]:
%%bash

curl -i http://192.168.0.116:5000/query-example?language=Python

curl -i http://192.168.0.116:5000/method \
-X POST

HTTP/1.0 200 OK
Content-Type: text/html; charset=utf-8
Content-Length: 39
Server: Werkzeug/2.0.2 Python/3.8.10
Date: Fri, 15 Oct 2021 14:27:20 GMT

<h1>The language value is: Python </h1>HTTP/1.0 200 OK
Content-Type: text/html; charset=utf-8
Content-Length: 27
Server: Werkzeug/2.0.2 Python/3.8.10
Date: Fri, 15 Oct 2021 14:27:20 GMT

You've used a Post request.

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    39  100    39    0     0  13000      0 --:--:-- --:--:-- --:--:-- 13000
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    27  100    27    0     0  13500      0 --:--:-- --:--:-- --:--:-- 27000


In [3]:
%%file restful.py
from flask import Flask
from flask_restful import Resource, Api

app = Flask(__name__)
api = Api(app)

class Student(Resource):
    def get(self, name):
        return {'student': name}
    
api.add_resource(Student, '/student/<string:name>')

if __name__=='__main__':
    app.run(host='0.0.0.0', debug=True)

Writing restful.py


In [4]:
%%bash

curl -i http://192.168.0.116:5000/student/Meng

HTTP/1.0 200 OK
Content-Type: application/json
Content-Length: 26
Server: Werkzeug/2.0.2 Python/3.8.10
Date: Fri, 15 Oct 2021 14:28:02 GMT

{
    "student": "Meng"
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    26  100    26    0     0  13000      0 --:--:-- --:--:-- --:--:-- 13000


In [5]:
%%file change.py
from flask import Flask
from flask import jsonify

app = Flask(__name__)

def change(amount):
    # calculate the resultant change and store the result (res)
    res = []
    coins = [1, 5, 10, 25] # value of pennies, nickels, dimes, quarters
    coin_lookup = {25: "quarters", 10: "dimes", 5: "nickels", 1: "pennies"}
    # divide the amount*100 (the amount in cents) by a coin value
    # record the number of coins that evenly divide and the remainder
    coin = coins.pop()
    num, rem = divmod(int(amount*100), coin)
    # append the coin type and number of coins that had no remainder
    res.append({num:coin_lookup[coin]})
    # while there is still some remainder, continue adding coins to the result
    while rem > 0:
        coin = coins.pop()
        num, rem = divmod(rem, coin)
        if num:
            if coin in coin_lookup:
                res.append({num:coin_lookup[coin]})
    return res

@app.route('/')
def hello():
    """Return a friendly HTTP greeting."""
    print('I am inside hello world')
    return 'Hello World! I can make change at route: /change'

@app.route('/change/<dollar>/<cents>')
def changeroute(dollar, cents):
    print(f'Make change for {dollar}.{cents}')
    amount = f'{dollar}.{cents}'
    result = change(float(amount))
    return jsonify(result)

if __name__=='__main__':
    app.run(host='0.0.0.0', debug=True)

Writing change.py


In [6]:
%%file test_with_pytest.py
from change import change

def test_change():
    assert [{5: 'quarters'}, {1: 'nickels'}, {4: 'pennies'}] == change(1.34)

Writing test_with_pytest.py


In [7]:
%%bash

pytest
curl -i http://192.168.0.116:5000/change/1/34

HTTP/1.0 200 OK
Content-Type: application/json
Content-Length: 90
Server: Werkzeug/2.0.2 Python/3.8.10
Date: Fri, 15 Oct 2021 14:28:32 GMT

[
  {
    "5": "quarters"
  }, 
  {
    "1": "nickels"
  }, 
  {
    "4": "pennies"
  }
]


bash: line 2: pytest: command not found
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    90  100    90    0     0  90000      0 --:--:-- --:--:-- --:--:-- 90000


In [8]:
%%file query.py
from flask import Flask, request

app = Flask(__name__)

@app.route('/query-example')
def query_example():
    language = request.args.get('language')
    framework = request.args['framework']
    website = request.args.get('website')
    return """
        <h1>The language value is: {}</h1>
        <h1>The framework value is: {}</h1>
        <h1>The website values is {}</h1>
        """.format(language, framework, website)

@app.route('/form-example', methods=['Get', 'Post'])
def form_example():
    if request.method == 'POST':
        language = request.form.get('language')
        framework = request.form.get('framework')
        return """
            <h1>The language value is: {}</h1>
            <h1>The framework value is: {}</h1>
            """.format(language, framework)
    else:
        language = request.args.get('language')
        framework = request.args['framework']
        return """
            <h1>The language value is: {}</h1>
            <h1>The framework value is: {}
            """.format(language, framework)
    
@app.route('/json-example', methods=['POST'])
def json_example():
    request_data = request.get_json()
    
    if request_data:
        if 'language' in request_data:
            language = request_data['language']
        
        if 'framework' in request_data:
            framework = request_data['framework']
    
    return """
        <h1>The language value is: {}</h1>
        <h1>The framework value is: {}
        """.format(language, framework)    

if __name__=='__main__':
    app.run(host='0.0.0.0', debug=True)

Writing query.py


In [9]:
%%bash

curl -i "http://192.168.0.116:5000/query-example?language=Python&framework=Flask&website=meng"

curl -i "http://192.168.0.116:5000/form-example?language=Python&framework=Flask"

curl -i "http://192.168.0.116:5000/form-example" \
-X POST \
-F 'language=Python' \
-F 'framework=Flask'

curl -i "http://192.168.0.116:5000/json-example" \
-H 'Content-Type: application/json' \
-X POST \
-d '{"language":"Python", "framework":"Flask"}'

HTTP/1.0 200 OK
Content-Type: text/html; charset=utf-8
Content-Length: 147
Server: Werkzeug/2.0.2 Python/3.8.10
Date: Fri, 15 Oct 2021 14:28:59 GMT


        <h1>The language value is: Python</h1>
        <h1>The framework value is: Flask</h1>
        <h1>The website values is meng</h1>
        HTTP/1.0 200 OK
Content-Type: text/html; charset=utf-8
Content-Length: 110
Server: Werkzeug/2.0.2 Python/3.8.10
Date: Fri, 15 Oct 2021 14:28:59 GMT


            <h1>The language value is: Python</h1>
            <h1>The framework value is: Flask
            HTTP/1.0 200 OK
Content-Type: text/html; charset=utf-8
Content-Length: 115
Server: Werkzeug/2.0.2 Python/3.8.10
Date: Fri, 15 Oct 2021 14:28:59 GMT


            <h1>The language value is: Python</h1>
            <h1>The framework value is: Flask</h1>
            HTTP/1.0 200 OK
Content-Type: text/html; charset=utf-8
Content-Length: 98
Server: Werkzeug/2.0.2 Python/3.8.10
Date: Fri, 15 Oct 2021 14:28:59 GMT


        <h1>The language value i

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   147  100   147    0     0  73500      0 --:--:-- --:--:-- --:--:--  143k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   110  100   110    0     0  55000      0 --:--:-- --:--:-- --:--:--  107k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   367  100   115  100   252  23000  50400 --:--:-- --:--:-- --:--:-- 73400
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   140  100    98  100    42  49000  21000 --:--:-- --:--:-- --:--:-- 70000


In [10]:
%%file home.py
from flask import Blueprint

home_bp = Blueprint('home', __name__)

@home_bp.route('/hello/')
def hello():
    return "Hello from Home Page", 200

Writing home.py


In [11]:
%%file increment.py
from flask import Blueprint, jsonify

increment_bp = Blueprint('increment', __name__)

@increment_bp.route('/<int:number>')
def increment(number):
    return jsonify(list(range(number))), 200

Writing increment.py


In [12]:
%%file app.py
from flask import Flask
from home import home_bp
from increment import increment_bp

app = Flask(__name__)

app.register_blueprint(home_bp, url_prefix='/home')
app.register_blueprint(increment_bp, url_prefix='/increment')

if __name__=='__main__':
    app.run(host='0.0.0.0', debug=True)

Writing app.py


In [13]:
%%bash

curl -i "http://192.168.0.116:5000/home/hello/"

curl -i "http://192.168.0.116:5000/increment/5"

HTTP/1.0 200 OK
Content-Type: text/html; charset=utf-8
Content-Length: 20
Server: Werkzeug/2.0.2 Python/3.8.10
Date: Fri, 15 Oct 2021 14:31:02 GMT

Hello from Home PageHTTP/1.0 200 OK
Content-Type: application/json
Content-Length: 32
Server: Werkzeug/2.0.2 Python/3.8.10
Date: Fri, 15 Oct 2021 14:31:02 GMT

[
  0, 
  1, 
  2, 
  3, 
  4
]


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    20  100    20    0     0  10000      0 --:--:-- --:--:-- --:--:-- 10000
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    32  100    32    0     0  16000      0 --:--:-- --:--:-- --:--:-- 16000


In [47]:
%%file log.py
from flask import Flask, abort

app = Flask(__name__)

@app.before_request
def before():
    print("This is executed BEFORE each request.")
    
@app.route('/logs')
def logs():
    app.logger.debug('This is a DEBUG message')
    app.logger.info('This is an INFO message')
    app.logger.warning('This is a WARNING message')
    app.logger.error('This is an ERROR message')
    return "Hello World!", 200

if __name__=='__main__':
    app.run(host='0.0.0.0', debug=True)

Overwriting log.py


In [48]:
%%bash

curl -i "http://192.168.0.116:5000/logs"

HTTP/1.0 200 OK
Content-Type: text/html; charset=utf-8
Content-Length: 12
Server: Werkzeug/2.0.2 Python/3.8.10
Date: Fri, 15 Oct 2021 15:34:11 GMT

Hello World!

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    12  100    12    0     0   3000      0 --:--:-- --:--:-- --:--:--  3000


In [49]:
%%bash

rm -rf *.py